In [ ]:
# The following should reload files if we change them on disk
%load_ext autoreload
%aimport econsim.work, econsim.resources, econsim.makeragent, econsim.designer, econsim.producer, econsim.serviceprovider, econsim.utils, econsim.distmakingmodel, econsim.market
%autoreload 1

from pdb import set_trace

In [ ]:
# The agent platform
import mesa
from econsim.distmakingmodel import DistMakingModel
from econsim.utils import generate_ranges, process_batch_results, gen_stats

range_weights, _price_weight, _quality_ratio, range_living_cost, range_threshold, range_resources = generate_ranges(2)

params = {"designers": 20, 
          "producers": 20, 
          "initial_wealth":10, 
          "weights": range_weights, 
          "living_cost": range_living_cost, 
          "threshold": range_threshold,
          "resources_amount": range_resources
          }

iterations = 1
max_steps=100


results = mesa.batch_run(
    DistMakingModel,
    parameters=params,
    iterations=iterations,
    # max_steps=max_steps-1, # steps start from 0
    max_steps=max_steps,
    number_processes=1,
    data_collection_period=-1,
    display_progress=True,
)


In [ ]:
ag_res_df, ag_glob_df, product_results_df = process_batch_results(results)

stats = gen_stats(max_steps, range_weights, range_living_cost, range_threshold, ag_res_df)

stats

In [ ]:
# Data visualization tools.
import matplotlib.pyplot as plt
import seaborn as sns


for pw,qw,sw in range_weights:
    for lc in range_living_cost:
        for t in range_threshold:
            
            plt.figure()
            data=ag_res_df.loc[ (ag_res_df["price_weight"] == pw) & 
                (ag_res_df["quality_weight"] == qw) & 
                (ag_res_df["sustainability_weight"] == sw) & 
                (ag_res_df["living_cost"] == lc) & 
                (ag_res_df["threshold"] == t)
            ]
            g = sns.lineplot(
                data=data,
                x="Step",
                y="Alive",
                hue="AgentType", palette="tab10",
                errorbar=None,
            )
            g.figure.set_size_inches(8, 4)
            g.set(
                title=f"Average Alive, price_weight {pw}, living cost {lc}, threshold {t}",
                ylabel="Alive",
            )
            g

In [ ]:
# import matplotlib.pyplot as plt

for ag_type in ["Designer", "Producer"]:
    for facet in ["Skill", "Fee"]:
        for pw,qw,sw in range_weights:
            for lc in range_living_cost:
                for t in range_threshold:

                    plt.figure()
                    data=ag_res_df.loc[ (ag_res_df["AgentType"] == ag_type) &
                        (ag_res_df["price_weight"] == pw) & 
                        (ag_res_df["quality_weight"] == qw) & 
                        (ag_res_df["sustainability_weight"] == sw) & 
                        (ag_res_df["living_cost"] == lc) & 
                        (ag_res_df["threshold"] == t)
                    ]

                    g = sns.lineplot(
                        data=data,
                        x="Step", y="Wealth", hue=facet, style=facet, palette="tab10",
                        # errorbar='sd',
                        errorbar=None,
                    )
                    g.figure.set_size_inches(8, 4)
                    g.set(
                        title=f"Average wealth {ag_type}s per {facet} level, weights {pw},{qw},{sw}, living cost {lc}, threshold {t}",
                        ylabel="Wealth",
                    )
                    g

In [ ]:
g = sns.lineplot(
    data=ag_res_df, x="Step", y="Wealth", hue="AgentType", palette="tab10",
    errorbar=None,
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average wealth",
    ylabel="Wealth",
)

In [ ]:
g = sns.lineplot(
    data=ag_glob_df, x="Step", y="Gini",
    errorbar=None,
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average Gini coefficient",
    ylabel="Gini coeff",
)

In [ ]:
g = sns.lineplot(
    data=product_results_df, x="Step", y="Designs in Progress"
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average Designs in Progress",
    ylabel="Design in Progress",
)

In [ ]:
g = sns.lineplot(
    data=product_results_df, x="Step", y="Realized Designs"
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average Realized Designs",
    ylabel="Realized Designs",
)

In [ ]:
g = sns.lineplot(
    data=product_results_df, x="Step", y="Products in Progress"
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average Products in Progress",
    ylabel="Products in Progress",
)

In [ ]:
g = sns.lineplot(
    data=product_results_df, x="Step", y="On-sale Products"
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average On-sale Products",
    ylabel="On-sale Products",
)

In [ ]:
g = sns.lineplot(
    data=product_results_df, x="Step", y="Sold Products"
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average Sold Products",
    ylabel="Sold Products",
)